In [1]:
import pandas as pd
import numpy as np
import keras
from keras import layers
from sklearn.metrics import (confusion_matrix, precision_score, auc,
                             roc_curve, recall_score, classification_report, f1_score)
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import random
import scipy.stats as stats
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import csv
from sklearn.cluster import KMeans
from sklearn_som.som import SOM
from sklearn import svm
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn import tree

# Dados NSL-KDD

In [2]:
data = pd.read_csv(r"C:\Users\luang\Downloads\BRACIS\UNSW_2018_IoT_Botnet_Full5p.csv")
x = data.drop (['category', 'subcategory', 'attack'], axis = 1)
y = data ['attack']

In [3]:
data = x.to_numpy()
data_class = y.to_numpy()

# Abordagem Autoencoder-KNN

In [4]:
epochs = 50
batch_size = 256
optimizer = Adam(learning_rate=0.0001)
activation = "relu"
neighbors = 11

num_folds = 10

kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)

fold_no = 1
count = 0

#Métricas KNN
matrix_knn = np.zeros((num_folds,2,2))
accuracy_knn = np.zeros(num_folds)
recall_knn = np.zeros(num_folds)
precision_knn = np.zeros(num_folds)
f1_knn = np.zeros(num_folds)

#Métricas Decision Tree
matrix_dt = np.zeros((num_folds,2,2))
accuracy_dt = np.zeros(num_folds)
recall_dt = np.zeros(num_folds)
precision_dt = np.zeros(num_folds)
f1_dt = np.zeros(num_folds)

#Métricas SVM
matrix_svm = np.zeros((num_folds,2,2))
accuracy_svm = np.zeros(num_folds)
recall_svm = np.zeros(num_folds)
precision_svm = np.zeros(num_folds)
f1_svm = np.zeros(num_folds)

for train, test in kfold.split(data, data_class):
    
    #####################################################
    ####Pré-Processamento
    
    #Separação dos Folds de Treinamento e Teste
    dataTrain = data[train]
    dataTrain_class = data_class[train]
    modelTest = data[test]
    modelTest_class = data_class[test]
    
    #Conta Quantidade de dados
    Train = dataTrain.shape[1]
    noTrain = np.count_nonzero(dataTrain_class == 0)
    anTrain = np.count_nonzero(dataTrain_class == 1)

    #Separa dados normais e anômalos
    normalTrain = dataTrain[np.where(dataTrain_class == 0)]
    anomalyTrain = dataTrain[np.where(dataTrain_class == 1)]
    
    #Divide dados para treinamento
    porcen = 1
    j = int(noTrain * porcen)
    
    #Índices escolhidos aleatoriamente
    numbers_knn_normal = np.array(random.sample(range(0, noTrain),j))
    numbers_knn_anomaly = np.array(random.sample(range(0,anTrain),j))
    numbers_autoencoder_anomaly = np.array(list(set(np.arange(0, anTrain)) - set(numbers_knn_anomaly)))
    
    #Dados KNN
    knnTrain_normal = normalTrain[numbers_knn_normal]
    knnTrain_anomaly = anomalyTrain[numbers_knn_anomaly]
    knnTrain = np.concatenate((knnTrain_normal, knnTrain_anomaly), axis=0)
    knnTrain_class = np.concatenate((np.zeros(j), np.ones(j)))
    
    #Dados Autoencoder
    autoencoderTrain = anomalyTrain[numbers_autoencoder_anomaly]
    autoencoderTrain_class = np.ones(autoencoderTrain.shape[0])
    
    #Normalização
    scaler = MinMaxScaler()
    scaler.fit(knnTrain)
    autoencoderTrain = scaler.transform(autoencoderTrain)
    knnTrain = scaler.transform(knnTrain)
    modelTest = scaler.transform(modelTest)
    
    #####################################################
    ####Autoencoder
    #Modela as camadas do Autoencoder
    var = autoencoderTrain.shape[1]
    input_vector = keras.Input(shape=(var,))
    en1 = layers.Dense(int(var/2), activation=activation)(input_vector)
    en2 = layers.Dense(int(var/4), activation=activation)(en1)
    de1 = layers.Dense(int(var/2), activation=activation)(en2)
    de2 = layers.Dense(var, activation=activation)(de1)

    #Gera o modelo
    autoencoder = keras.Model(input_vector, de2)
    
    encoder = keras.Model(input_vector, en2)
    encoded_input = keras.Input(shape=(int(var/4),))
    decoder_layer = autoencoder.layers[-2]
    decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

    #Compila o modelo
    autoencoder.compile(optimizer=optimizer, loss="mean_squared_error")

    #Treina o modelo
    history = autoencoder.fit(autoencoderTrain, autoencoderTrain, epochs = epochs, batch_size = batch_size, shuffle = True)
    
    
    #####################################################
    ####KNN
    ###Treinamento KNN
    pred_knn = autoencoder.predict(knnTrain)
    mse_knn = np.mean(np.power(knnTrain - pred_knn, 2), axis=1)
    error_df_knn = pd.DataFrame({'reconstruction_error': mse_knn,
                                 'true_class': knnTrain_class})
    error_knn = error_df_knn.to_numpy()
    encoder_layer = encoder.predict(knnTrain)
    error_encoder = np.append(encoder_layer, error_knn[:,0].reshape(-1,1), axis=1)
    neigh = KNeighborsClassifier(n_neighbors=neighbors)
    neigh.fit(error_encoder, error_knn[:,1])
    
    #Teste do Modelo
    pred_knn = autoencoder.predict(modelTest)
    mse_knn = np.mean(np.power(modelTest - pred_knn, 2), axis=1)
    error_df_knn = pd.DataFrame({'reconstruction_error': mse_knn,
                                'true_class': modelTest_class})
    error_knn = error_df_knn.to_numpy()
    encoder_layer = encoder.predict(modelTest)
    error_encoder = np.append(encoder_layer, error_knn[:,0].reshape(-1,1), axis=1)
    predict_knn = neigh.predict(error_encoder)
    
    #Métricas
    accuracy_knn[count] = accuracy_score(error_knn[:,1], predict_knn[:])
    recall_knn[count] = recall_score(error_knn[:,1], predict_knn[:])
    precision_knn[count] = precision_score(error_knn[:,1], predict_knn[:])
    f1_knn[count] = f1_score(error_knn[:,1], predict_knn[:])
    matrix_knn[count] = confusion_matrix(error_knn[:,1], predict_knn[:])
    
      
    #####################################################
    ####Decision Tree
    ###Treinamento Decisition Tree 
    pred_dt = autoencoder.predict(knnTrain)
    mse_dt = np.mean(np.power(knnTrain - pred_dt, 2), axis=1)
    error_df_dt = pd.DataFrame({'reconstruction_error': mse_dt,
                                     'true_class': knnTrain_class})
    error_dt = error_df_dt.to_numpy()
    encoder_layer = encoder.predict(knnTrain)
    error_encoder = np.append(encoder_layer, error_dt[:,0].reshape(-1,1), axis=1)
    dt = tree.DecisionTreeClassifier()
    dt.fit(error_encoder, error_dt[:,1])

    #Teste
    pred_dt = autoencoder.predict(modelTest)
    mse_dt = np.mean(np.power(modelTest - pred_dt, 2), axis=1)
    error_df_dt = pd.DataFrame({'reconstruction_error': mse_dt,
                                     'true_class': modelTest_class})
    error_dt = error_df_dt.to_numpy()
    encoder_layer = encoder.predict(modelTest)
    error_encoder = np.append(encoder_layer,error_dt[:,0].reshape(-1,1), axis=1)
    predict_dt = dt.predict(error_encoder)

    #Métricas
    accuracy_dt[count] = accuracy_score(error_dt[:,1], predict_dt[:])
    recall_dt[count] = recall_score(error_dt[:,1], predict_dt[:])
    precision_dt[count] = precision_score(error_dt[:,1], predict_dt[:])
    f1_dt[count] = f1_score(error_dt[:,1], predict_dt[:])
    matrix_dt[count] = confusion_matrix(error_dt[:,1], predict_dt[:])
       
    #####################################################
    ####SVM
    #Treinamento SVM
    pred_svm = autoencoder.predict(knnTrain)
    mse_svm = np.mean(np.power(knnTrain - pred_svm, 2), axis=1)
    error_df_svm = pd.DataFrame({'reconstruction_error': mse_svm,
                                     'true_class': knnTrain_class})
    error_svm = error_df_svm.to_numpy()
    encoder_layer = encoder.predict(knnTrain)
    error_encoder = np.append(encoder_layer, error_svm[:,0].reshape(-1,1), axis=1)
    clf = svm.SVC(kernel='rbf')
    clf.fit(error_encoder, error_svm[:,1])
    
    #Teste
    pred_svm = autoencoder.predict(modelTest)
    mse_svm = np.mean(np.power(modelTest - pred_svm, 2), axis=1)
    error_df_svm = pd.DataFrame({'reconstruction_error': mse_svm,
                                     'true_class': modelTest_class})
    error_svm = error_df_svm.to_numpy()
    encoder_layer = encoder.predict(modelTest)
    error_encoder = np.append(encoder_layer,error_svm[:,0].reshape(-1,1), axis=1)
    predict_svm = clf.predict(error_encoder)
    
    #Métricas
    accuracy_svm[count] = accuracy_score(error_svm[:,1], predict_svm[:])
    recall_svm[count] = recall_score(error_svm[:,1], predict_svm[:])
    precision_svm[count] = precision_score(error_svm[:,1], predict_svm[:])
    f1_svm[count] = f1_score(error_svm[:,1], predict_svm[:])
    matrix_svm[count] = confusion_matrix(error_svm[:,1], predict_svm[:])

    count = count + 1
    fold_no = fold_no + 1

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
12894/12894 [==============================] - 25s 2ms/step - loss: 0.0095
Epoch 2/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0015
Epoch 3/50
12894/12894 [==============================] - 16s 1ms/step - loss: 0.0012
Epoch 4/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0011
Epoch 5/50
12894/12894 [==========

12894/12894 [==============================] - 17s 1ms/step - loss: 0.0042
Epoch 3/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0037
Epoch 4/50
12894/12894 [==============================] - 20s 2ms/step - loss: 0.0035
Epoch 5/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0035
Epoch 6/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0034
Epoch 7/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0034
Epoch 8/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0034
Epoch 9/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0034
Epoch 10/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0034
Epoch 11/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0034
Epoch 12/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0034
Epoch 13/50
12894/12894 [==============================] - 17s

12894/12894 [==============================] - 17s 1ms/step - loss: 0.0053
Epoch 15/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0053
Epoch 16/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0053
Epoch 17/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0053
Epoch 18/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0053
Epoch 19/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0053
Epoch 20/50
12894/12894 [==============================] - 23s 2ms/step - loss: 0.0053
Epoch 21/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0053
Epoch 22/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0053
Epoch 23/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0053
Epoch 24/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0053
Epoch 25/50
12894/12894 [==============================

12894/12894 [==============================] - 27s 2ms/step - loss: 0.0055
Epoch 27/50
12894/12894 [==============================] - 28s 2ms/step - loss: 0.0055
Epoch 28/50
12894/12894 [==============================] - 27s 2ms/step - loss: 0.0055
Epoch 29/50
12894/12894 [==============================] - 29s 2ms/step - loss: 0.0055
Epoch 30/50
12894/12894 [==============================] - 28s 2ms/step - loss: 0.0055
Epoch 31/50
12894/12894 [==============================] - 28s 2ms/step - loss: 0.0055
Epoch 32/50
12894/12894 [==============================] - 28s 2ms/step - loss: 0.0054
Epoch 33/50
12894/12894 [==============================] - 29s 2ms/step - loss: 0.0054
Epoch 34/50
12894/12894 [==============================] - 20s 2ms/step - loss: 0.0054
Epoch 35/50
12894/12894 [==============================] - 19s 1ms/step - loss: 0.0054
Epoch 36/50
12894/12894 [==============================] - 24s 2ms/step - loss: 0.0054
Epoch 37/50
12894/12894 [==============================

12894/12894 [==============================] - 20s 2ms/step - loss: 0.0212
Epoch 38/50
12894/12894 [==============================] - 23s 2ms/step - loss: 0.0212
Epoch 39/50
12894/12894 [==============================] - 23s 2ms/step - loss: 0.0212
Epoch 40/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0212
Epoch 41/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0212
Epoch 42/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0212
Epoch 43/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0212
Epoch 44/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0212
Epoch 45/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0212
Epoch 46/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0212
Epoch 47/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0212
Epoch 48/50
12894/12894 [==============================

12894/12894 [==============================] - 20s 2ms/step - loss: 0.0161
Epoch 50/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0161
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing t

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
12894/12894 [==============================] - 26s 2ms/step - loss: 0.0236
Epoch 2/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0189
Epoch 3/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0186
Epoch 4/50
12894/12894 [==============================] - 18s 1ms/step - loss: 0.0186
Epoch 5/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0185
Epoch 6/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0185
Epoch 7/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0185: 0s 
Epoch 8/50
12894/12894 [==============================] - 17s 1ms/step - loss: 0.0185
Epoch 9

12894/12894 [==============================] - 25s 2ms/step - loss: 0.0175
Epoch 8/50
12894/12894 [==============================] - 26s 2ms/step - loss: 0.0175
Epoch 9/50
12894/12894 [==============================] - 27s 2ms/step - loss: 0.0175
Epoch 10/50
12894/12894 [==============================] - 21s 2ms/step - loss: 0.0175
Epoch 11/50
12894/12894 [==============================] - 19s 1ms/step - loss: 0.0175
Epoch 12/50
12894/12894 [==============================] - 19s 1ms/step - loss: 0.0175
Epoch 13/50
12894/12894 [==============================] - 21s 2ms/step - loss: 0.0175: 0s - loss: 
Epoch 14/50
12894/12894 [==============================] - 19s 2ms/step - loss: 0.0175
Epoch 15/50
12894/12894 [==============================] - 19s 1ms/step - loss: 0.0175
Epoch 16/50
12894/12894 [==============================] - 19s 1ms/step - loss: 0.0175
Epoch 17/50
12894/12894 [==============================] - 19s 1ms/step - loss: 0.0175
Epoch 18/50
12894/12894 [===================

In [5]:
print("Autoencoder - KNN:\n", np.mean(matrix_knn, axis=0), "\nAcurácia: ", np.mean(accuracy_knn), ' +/- ', np.std(accuracy_knn), "\nRecall: ", np.mean(recall_knn), ' +/- ', np.std(recall_knn), "\nPrecision: ", np.mean(precision_knn), ' +/- ', np.std(precision_knn), "\nF1: ", np.mean(f1_knn), ' +/- ', np.std(f1_knn))

print("\nAutoencoder - SVM:\n", np.mean(matrix_svm, axis=0), "\nAcurácia: ", np.mean(accuracy_svm), ' +/- ', np.std(accuracy_svm), "\nRecall: ", np.mean(recall_svm), ' +/- ', np.std(recall_svm), "\nPrecision: ", np.mean(precision_svm), ' +/- ', np.std(precision_svm), "\nF1: ", np.mean(f1_svm), ' +/- ', np.std(f1_svm))

print("\nAutoencoder - Decision Tree:\n", np.mean(matrix_dt, axis=0), "\nAcurácia: ", np.mean(accuracy_dt), ' +/- ', np.std(accuracy_dt), "\nRecall: ", np.mean(recall_dt), ' +/- ', np.std(recall_dt), "\nPrecision: ", np.mean(precision_dt), ' +/- ', np.std(precision_dt), "\nF1: ", np.mean(f1_dt), ' +/- ', np.std(f1_dt))

Autoencoder - KNN:
 [[4.700000e+01 7.000000e-01]
 [5.213900e+03 3.615906e+05]] 
Acurácia:  0.9857855585485501  +/-  0.008073759727249661 
Recall:  0.9857856140518784  +/-  0.008074699115840469 
Precision:  0.9999980629385773  +/-  2.159734509358166e-06 
F1:  0.9928242943582062  +/-  0.004103321018109197

Autoencoder - SVM:
 [[4.710000e+01 6.000000e-01]
 [8.159400e+03 3.586451e+05]] 
Acurácia:  0.9777567102715332  +/-  0.008027124446846166 
Recall:  0.9777554519870251  +/-  0.008027698286780684 
Precision:  0.9999983228401528  +/-  1.8618157456467052e-06 
F1:  0.9887351491378673  +/-  0.0041054234649395204

Autoencoder - Decision Tree:
 [[4.720000e+01 5.000000e-01]
 [2.545400e+03 3.642591e+05]] 
Acurácia:  0.9930601475863796  +/-  0.0030871189532411356 
Recall:  0.9930606073324025  +/-  0.003087782448965782 
Precision:  0.9999986281573159  +/-  1.8428806504927488e-06 
F1:  0.9965151313713371  +/-  0.0015554685626742743


In [6]:
Result = np.array(["KNN", "Acurácia", accuracy_knn, np.mean(accuracy_knn), np.std(accuracy_knn),"",
                                "Recall", recall_knn, np.mean(recall_knn), np.std(recall_knn), "",
                                "Precision", precision_knn, np.mean(precision_knn), np.std(precision_knn), "",
                                "F1", f1_knn, np.mean(f1_knn), np.std(f1_knn), "-----------------","-----------------",
                  "Decision Tree",  "Acurácia", accuracy_dt, np.mean(accuracy_dt), np.std(accuracy_dt),"",
                                "Recall", recall_dt, np.mean(recall_dt), np.std(recall_dt), "",
                                "Precision", precision_dt, np.mean(precision_dt), np.std(precision_dt), "",
                                "F1", f1_dt, np.mean(f1_dt), np.std(f1_dt), "-----------------","-----------------",
                  "SVM",  "Acurácia", accuracy_svm, np.mean(accuracy_svm), np.std(accuracy_svm),"",
                                "Recall", recall_svm, np.mean(recall_svm), np.std(recall_svm), "",
                                "Precision", precision_svm, np.mean(precision_svm), np.std(precision_svm), "",
                                "F1", f1_svm, np.mean(f1_svm), np.std(f1_svm), "","",])

np.savetxt('Result_Error+IL_IoT.txt', Result, delimiter = ',',fmt='%s')

C:\Users\luang\anaconda3\envs\keras_env\lib\site-packages\ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == '':


In [7]:
Result_Matrix = np.array(["KNN", matrix_knn, np.mean(matrix_knn, axis=0), np.std(matrix_knn, axis=0), "-----------------","-----------------",
                          "Decision Tree", matrix_dt, np.mean(matrix_dt, axis=0), np.std(matrix_dt, axis=0), "-----------------","-----------------",
                          "SVM", matrix_svm, np.mean(matrix_svm, axis=0), np.std(matrix_svm, axis=0)])

np.savetxt('Result_Error+IL_Matrix_IoT.txt', Result_Matrix, delimiter = ',',fmt='%s')

C:\Users\luang\anaconda3\envs\keras_env\lib\site-packages\ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until
